In [341]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [342]:
pd.options.display.max_columns = 200

In [343]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [344]:
ID = test.ID

In [345]:
train.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,SeedingSowingTransplanting,SeedlingsPerPit,NursDetFactor,TransDetFactor,TransplantingIrrigationHours,TransplantingIrrigationSource,TransplantingIrrigationPowerSource,TransIrriCost,StandingWater,OrgFertilizers,Ganaura,CropOrgFYM,PCropSolidOrgFertAppMethod,NoFertilizerAppln,CropbasalFerts,BasalDAP,BasalUrea,MineralFertAppMethod,FirstTopDressFert,1tdUrea,1appDaysUrea,2tdUrea,2appDaysUrea,MineralFertAppMethod.1,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
0,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-06-27,2022-07-21,2.0,CalendarDate IrrigWaterAvailability SeedAvaila...,CalendarDate SeedlingAge RainArrival IrrigWate...,5.0,Boring,Electric,200.0,2.0,NaN,NaN,NaN,NaN,2,Urea,NaN,20.0,Broadcasting,Urea,15.0,18.0,NaN,NaN,Broadcasting,machine,2022-11-16,NaN,2022-11-16,machine,30,40,plowed_in_soil,0.312500,600
1,ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-06-20,2022-07-20,2.0,CalendarDate PreMonsoonShowers IrrigWaterAvail...,CalendarDate SeedlingAge RainArrival IrrigWate...,5.0,Boring,Electric,125.0,3.0,NaN,NaN,NaN,NaN,2,DAP Urea,15.0,10.0,Broadcasting,Urea,20.0,39.0,NaN,NaN,Broadcasting,hand,2022-11-25,3.0,2022-12-24,machine,24,10,plowed_in_soil,0.312500,600
2,ID_1FJY2CRIMLZZ,Gaya,Gurua,10,10,TractorPlough FourWheelTracRotavator,2022-06-30,6,Manual_PuddledRandom,2022-06-20,2022-08-13,2.0,PreMonsoonShowers IrrigWaterAvailability Labou...,SeedlingAge IrrigWaterAvailability LaborAvaila...,4.0,Boring,Electric,80.0,2.0,Ganaura FYM,1.0,1.0,SoilApplied,2,DAP,4.0,NaN,SoilApplied,Urea,5.0,65.0,NaN,NaN,RootApplication,hand,2022-12-12,480.0,2023-01-11,machine,30,10,plowed_in_soil,0.148148,225
3,ID_I3IPXS4DB7NE,Gaya,Gurua,15,15,TractorPlough FourWheelTracRotavator,2022-06-16,6,Manual_PuddledRandom,2022-06-17,2022-07-17,2.0,CalendarDate PreMonsoonShowers IrrigWaterAvail...,CalendarDate SeedlingAge RainArrival IrrigWate...,NaN,NaN,NaN,NaN,NaN,Ganaura,1.0,NaN,SoilApplied,2,DAP Urea,6.0,3.0,Broadcasting,Urea,5.0,5.0,NaN,NaN,Broadcasting,hand,2022-12-02,240.0,2022-12-29,hand,26,10,plowed_in_soil,0.222222,468
4,ID_4T8YQWXWHB4A,Nalanda,Noorsarai,60,60,TractorPlough WetTillagePuddling,2022-07-19,4,Manual_PuddledRandom,2022-06-21,2022-07-20,2.0,CalendarDate IrrigWaterAvailability SeedAvaila...,SeedlingAge RainArrival IrrigWaterAvailability...,9.0,Boring,Electric,300.0,2.0,NaN,NaN,NaN,NaN,2,Urea DAP,15.0,30.0,Broadcasting,Urea,30.0,26.0,NaN,NaN,Broadcasting,machine,2022-11-30,NaN,2022-12-02,machine,24,40,plowed_in_soil,0.468750,550


 # Feature Engineering for Numarical values

In [346]:
def stats_by_feat(train,test,feat_to_stat,feat_to_group):
    for df in (train,test):
        df[feat_to_stat+feat_to_group+'std'] = df['BS'].map(df.groupby('BS')['load'].aggregate('std'))
        df[feat_to_stat+feat_to_group+'median'] = df['BS'].map(df.groupby('BS')['load'].aggregate('median'))
        df[feat_to_stat+feat_to_group+'min'] = df['BS'].map(df.groupby('BS')['load'].aggregate('min'))
        df[feat_to_stat+feat_to_group+'max'] = df['BS'].map(df.groupby('BS')['load'].aggregate('max'))
        df[feat_to_stat+feat_to_group+'mean'] = df['BS'].map(df.groupby('BS')['load'].aggregate('mean'))

In [347]:
def stats_with_features(train,test,func,*feat):
    for df in (train,test):
        name = ''
        for f in feat:
            name += f
        df[name+func] = df[list(feat)].agg(func,axis=1)
        

In [348]:
def month(*features):
    for df in (train,test):
        for feat in features:
            df[feat+'month'] = df[feat].dt.month

In [349]:
def stats_by_feat(train,test,feat_to_stat,feat_to_group,func):
    for df in (train,test):
        df[feat_to_stat+feat_to_group+func] = df[feat_to_group].map(df.groupby(feat_to_group)[feat_to_stat].aggregate(func))

In [350]:
def day_Of_year(*features):
    for df in (train,test):
        for feat in features:
            df[feat+'_day'] = df[feat].dt.day_of_year

    

In [351]:
def to_date(*feats):
    for df in (train,test):
        for feature in feats:
            df[feature] = pd.to_datetime(df[feature])

In [352]:
def count_encoder(features):
    for feature in features:
        for df in (train,test):
            df[feature] = df[feature].map(df[feature].value_counts(normalize=True))


In [353]:
to_date('CropTillageDate','RcNursEstDate','SeedingSowingTransplanting','Harv_date')
day_Of_year('CropTillageDate','RcNursEstDate','SeedingSowingTransplanting','Harv_date')


In [354]:
stats_with_features(train,test, 'mean', '1tdUrea','Acre')
stats_with_features(train,test, 'mean', '2tdUrea','1tdUrea')

    

In [355]:
# for df in (train,test):
#     df.drop(columns='CropTillageDatemonth',inplace=True)

In [356]:
train.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,SeedingSowingTransplanting,SeedlingsPerPit,NursDetFactor,TransDetFactor,TransplantingIrrigationHours,TransplantingIrrigationSource,TransplantingIrrigationPowerSource,TransIrriCost,StandingWater,OrgFertilizers,Ganaura,CropOrgFYM,PCropSolidOrgFertAppMethod,NoFertilizerAppln,CropbasalFerts,BasalDAP,BasalUrea,MineralFertAppMethod,FirstTopDressFert,1tdUrea,1appDaysUrea,2tdUrea,2appDaysUrea,MineralFertAppMethod.1,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield,CropTillageDate_day,RcNursEstDate_day,SeedingSowingTransplanting_day,Harv_date_day,1tdUreaAcremean,2tdUrea1tdUreamean
0,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-06-27,2022-07-21,2.0,CalendarDate IrrigWaterAvailability SeedAvaila...,CalendarDate SeedlingAge RainArrival IrrigWate...,5.0,Boring,Electric,200.0,2.0,NaN,NaN,NaN,NaN,2,Urea,NaN,20.0,Broadcasting,Urea,15.0,18.0,NaN,NaN,Broadcasting,machine,2022-11-16,NaN,2022-11-16,machine,30,40,plowed_in_soil,0.312500,600,201,178.0,202,320,7.656250,15.0
1,ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-06-20,2022-07-20,2.0,CalendarDate PreMonsoonShowers IrrigWaterAvail...,CalendarDate SeedlingAge RainArrival IrrigWate...,5.0,Boring,Electric,125.0,3.0,NaN,NaN,NaN,NaN,2,DAP Urea,15.0,10.0,Broadcasting,Urea,20.0,39.0,NaN,NaN,Broadcasting,hand,2022-11-25,3.0,2022-12-24,machine,24,10,plowed_in_soil,0.312500,600,199,171.0,201,329,10.156250,20.0
2,ID_1FJY2CRIMLZZ,Gaya,Gurua,10,10,TractorPlough FourWheelTracRotavator,2022-06-30,6,Manual_PuddledRandom,2022-06-20,2022-08-13,2.0,PreMonsoonShowers IrrigWaterAvailability Labou...,SeedlingAge IrrigWaterAvailability LaborAvaila...,4.0,Boring,Electric,80.0,2.0,Ganaura FYM,1.0,1.0,SoilApplied,2,DAP,4.0,NaN,SoilApplied,Urea,5.0,65.0,NaN,NaN,RootApplication,hand,2022-12-12,480.0,2023-01-11,machine,30,10,plowed_in_soil,0.148148,225,181,171.0,225,346,2.574074,5.0
3,ID_I3IPXS4DB7NE,Gaya,Gurua,15,15,TractorPlough FourWheelTracRotavator,2022-06-16,6,Manual_PuddledRandom,2022-06-17,2022-07-17,2.0,CalendarDate PreMonsoonShowers IrrigWaterAvail...,CalendarDate SeedlingAge RainArrival IrrigWate...,NaN,NaN,NaN,NaN,NaN,Ganaura,1.0,NaN,SoilApplied,2,DAP Urea,6.0,3.0,Broadcasting,Urea,5.0,5.0,NaN,NaN,Broadcasting,hand,2022-12-02,240.0,2022-12-29,hand,26,10,plowed_in_soil,0.222222,468,167,168.0,198,336,2.611111,5.0
4,ID_4T8YQWXWHB4A,Nalanda,Noorsarai,60,60,TractorPlough WetTillagePuddling,2022-07-19,4,Manual_PuddledRandom,2022-06-21,2022-07-20,2.0,CalendarDate IrrigWaterAvailability SeedAvaila...,SeedlingAge RainArrival IrrigWaterAvailability...,9.0,Boring,Electric,300.0,2.0,NaN,NaN,NaN,NaN,2,Urea DAP,15.0,30.0,Broadcasting,Urea,30.0,26.0,NaN,NaN,Broadcasting,machine,2022-11-30,NaN,2022-12-02,machine,24,40,plowed_in_soil,0.468750,550,200,172.0,201,334,15.234375,30.0


# Feature Engineering for Categorical features

### clustering and vectorizing words

In [357]:
tdfv = TfidfVectorizer()
landprep = pd.concat([train['LandPreparationMethod'],test['LandPreparationMethod']])
tdfv.fit(landprep)
lpmv = tdfv.transform(landprep)
km = KMeans(n_clusters=12)
train_land = km.fit_predict(lpmv)[:3870]
test_land = km.fit_predict(lpmv)[3870:]

In [365]:
train['LandPreparationMethod'] = train_land
test['LandPreparationMethod'] = test_land

In [366]:
train = train.select_dtypes(exclude='datetime64')
test = test.select_dtypes(exclude='datetime64')

In [367]:
train.drop(columns='ID',inplace=True)

In [368]:
count_encoder(train.select_dtypes(include='object'))

# Cross Validation

In [369]:
# model =  CatBoostRegressor()
model = LGBMRegressor()

In [370]:
kfold = KFold(n_splits=5,shuffle=True,random_state=69)

In [371]:
met = []
X = np.array(train.drop(columns='Yield'))
y = np.array(train['Yield'])
for train_index, test_index in kfold.split(X=X, y=y):
    X_train,y_train,X_test,_y_test = X[train_index], y[train_index], X[test_index], y[test_index]
    model.fit(X_train,y_train)
    mse = mean_squared_error(_y_test,model.predict(X_test),squared=False)
    met.append(mse)
 
np.array(met).mean()

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1566
[LightGBM] [Info] Number of data points in the train set: 3096, number of used features: 43
[LightGBM] [Info] Start training from score 601.696382
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001862 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1570
[LightGBM] [Info] Number of data points in the train set: 3096, number of used features: 43
[LightGBM] [Info] Start training from score 595.390181
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1566
[LightGBM] [Info] Number of data points in the train set: 3096, number of used features: 44
[LightGBM] [Info]

479.40020469001666

In [372]:
np.array(met).mean()

479.40020469001666

In [373]:
y=train['Yield']
train.drop(columns='Yield', inplace=True)

In [374]:
train

,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDepth,CropEstMethod,SeedlingsPerPit,NursDetFactor,TransDetFactor,TransplantingIrrigationHours,TransplantingIrrigationSource,TransplantingIrrigationPowerSource,TransIrriCost,StandingWater,OrgFertilizers,Ganaura,CropOrgFYM,PCropSolidOrgFertAppMethod,NoFertilizerAppln,CropbasalFerts,BasalDAP,BasalUrea,MineralFertAppMethod,FirstTopDressFert,1tdUrea,1appDaysUrea,2tdUrea,2appDaysUrea,MineralFertAppMethod.1,Harv_method,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,CropTillageDate_day,RcNursEstDate_day,SeedingSowingTransplanting_day,Harv_date_day,1tdUreaAcremean,2tdUrea1tdUreamean
0,0.308269,0.088630,45,40,2,5,0.864599,2.0,0.085451,0.239598,5.0,0.859654,0.785566,200.0,2.0,NaN,NaN,NaN,NaN,2,0.083650,NaN,20.0,0.830491,0.826588,15.0,18.0,NaN,NaN,0.970198,0.058915,NaN,0.012661,0.542119,30,40,0.993798,0.312500,201,178.0,202,320,7.656250,15.0
1,0.308269,0.219638,26,26,0,5,0.864599,2.0,0.156381,0.239598,5.0,0.859654,0.785566,125.0,3.0,NaN,NaN,NaN,NaN,2,0.445410,15.0,10.0,0.830491,0.826588,20.0,39.0,NaN,NaN,0.970198,0.941085,3.0,0.019121,0.542119,24,10,0.993798,0.312500,199,171.0,201,329,10.156250,20.0
2,0.147287,0.092506,10,10,2,6,0.864599,2.0,0.004189,0.007819,4.0,0.859654,0.785566,80.0,2.0,0.033136,1.0,1.0,0.663245,2,0.374525,4.0,NaN,0.164858,0.826588,5.0,65.0,NaN,NaN,0.010918,0.941085,480.0,0.006202,0.542119,30,10,0.993798,0.148148,181,171.0,225,346,2.574074,5.0
3,0.147287,0.092506,15,15,2,6,0.864599,2.0,0.156381,0.239598,NaN,NaN,NaN,NaN,NaN,0.510848,1.0,NaN,0.663245,2,0.445410,6.0,3.0,0.830491,0.826588,5.0,5.0,NaN,NaN,0.970198,0.941085,240.0,0.016537,0.457881,26,10,0.993798,0.222222,167,168.0,198,336,2.611111,5.0
4,0.308269,0.088630,60,60,5,4,0.864599,2.0,0.085451,0.051103,9.0,0.859654,0.785566,300.0,2.0,NaN,NaN,NaN,NaN,2,0.036393,15.0,30.0,0.830491,0.826588,30.0,26.0,NaN,NaN,0.970198,0.058915,NaN,0.009561,0.542119,24,40,0.993798,0.468750,200,172.0,201,334,15.234375,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865,0.308269,0.219638,28,28,0,6,0.053230,NaN,NaN,NaN,7.0,0.859654,0.785566,360.0,5.0,0.510848,1.0,NaN,0.663245,2,0.445410,18.0,12.0,0.830491,0.826588,12.0,45.0,NaN,NaN,0.970198,0.058915,NaN,0.006977,0.542119,25,40,0.993798,0.375000,183,180.0,197,333,6.187500,12.0
3866,0.253230,0.156848,5,5,0,5,0.864599,1.0,0.006981,0.004468,3.0,0.859654,0.213840,NaN,3.0,NaN,NaN,NaN,NaN,3,0.374525,6.0,NaN,0.004651,0.826588,6.0,45.0,6.0,6.0,0.970198,0.941085,4.0,0.008010,0.457881,30,10,0.993798,0.136364,187,189.0,211,283,3.068182,6.0
3867,0.308269,0.088630,50,45,1,5,0.864599,1.0,0.105836,0.012846,6.0,0.859654,0.785566,120.0,5.0,0.510848,1.0,NaN,0.332017,1,0.083650,NaN,24.0,0.830491,NaN,NaN,NaN,NaN,NaN,NaN,0.941085,1200.0,0.016279,0.457881,25,10,0.993798,0.375000,188,166.0,189,293,0.375000,NaN
3868,0.291214,0.129199,20,20,1,3,0.864599,4.0,0.034348,0.030438,2.0,0.099334,NaN,NaN,9.0,NaN,NaN,NaN,NaN,1,0.445410,7.0,10.0,0.830491,NaN,NaN,NaN,NaN,NaN,NaN,0.941085,100.0,0.021447,0.457881,25,10,0.993798,0.227273,193,179.0,205,313,0.227273,NaN


# Submission File

In [375]:
test.drop('ID',axis=1,inplace=True)

In [376]:
# Make predictions on the Zindi test set
model.fit(train,y)
preds = model.predict(test)
# Create submisiion file to be uploaded to Zindi for scoring
sub = pd.DataFrame({'ID': ID, 'Yield': preds})
sub.to_csv('submission-file.csv', index = False)

sub.head()

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1611
[LightGBM] [Info] Number of data points in the train set: 3870, number of used features: 44
[LightGBM] [Info] Start training from score 594.269251


,ID,Yield
0,ID_F9XXEXN2ADR2,558.899000
1,ID_SO3VW2X4QO93,378.619749
2,ID_UKUQ7JM8E894,449.567671
3,ID_QUISMWEZR2H4,289.078463
4,ID_25JGI455VKCZ,532.021535


In [377]:
pd.DataFrame({
    'name':model.feature_name_,
    'importance':model.feature_importances_
              }).sort_values('importance',ascending=False)

,name,importance
37,Acre,395
26,1appDaysUrea,362
13,TransIrriCost,233
39,RcNursEstDate_day,210
31,Harv_hand_rent,156
22,BasalUrea,139
32,Threshing_date,126
20,CropbasalFerts,125
38,CropTillageDate_day,123
40,SeedingSowingTransplanting_day,118
